In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import time
import pandas as pd

import re

## Get Companies

In [2]:
comps = pd.read_excel(r'C:\Users\Fabrice\Documents\GitHub\biz_sentiment\data\fraud\fraud.xlsx', sheet_name=1)
comps['search_term'] = comps['coname'].str.replace('�', "'")
comps['search_term'] = comps['search_term'].str.replace(' Company', "")
comps['search_term'] = comps['search_term'].str.replace(' Communication Systems', "")
comps['search_term'] = comps['search_term'].str.replace(' Corporation', "")
comps['search_term'] = comps['search_term'].str.replace(' Incorporated', "")
comps['search_term'] = comps['search_term'].str.replace(', Inc.', "")
comps['search_term'] = comps['search_term'].str.replace(', Inc', "")
comps['search_term'] = comps['search_term'].str.replace(' Inc.', "")
comps['search_term'] = comps['search_term'].str.replace(' Inc', "")
comps['search_term'] = comps['search_term'].str.replace(' PLC', "")
comps['search_term'] = comps['search_term'].str.replace(' S.A.', "")
comps

,permno,cusip,gvkey,conm,p_aaer,date_release,coname,fraud_date,search_term
0,80913,00819010,30697,AFFILIATED COMPUTER SERVICES,3182,2010-09-09,"Affiliated Computer Services, Inc.",1994-09-30,Affiliated Computer Services
1,14593,03783310,1690,APPLE INC,2880,2008-09-16,"Apple, Inc.",2001-09-30,Apple
2,81284,06367110,15580,BANK OF MONTREAL,2901,2008-11-18,Bank of Montreal,2006-01-31,Bank of Montreal
3,80193,07556Q88,29755,BEAZER HOMES USA INC,2884,2008-09-24,"Beazer Homes USA, Inc.",1999-12-31,Beazer Homes USA
4,78172,09182610,26012,BLACK BOX CORP,3073,2009-12-04,Black Box Corporation,1994-06-30,Black Box
5,86745,09228F10,65986,BLACKBERRY LTD,2937,2009-02-17,Research in Motion,1999-05-31,Research in Motion
6,87377,09534T50,126615,BLUE COAT SYSTEMS INC,2897,2008-11-12,"Blue Coat Systems, Inc.",1999-07-31,Blue Coat Systems
7,85963,11132010,66708,BROADCOM CORP,2827,2008-05-14,Broadcom Corporation,1998-06-30,Broadcom
8,86881,11162130,120774,BROCADE COMMUNICATIONS SYS,2961,2009-04-13,Brocade Communication Systems,2001-01-31,Brocade
9,90200,12680430,264708,CABELAS INC,3770,2016-04-26,Cabela's Incorporated,2012-03-31,Cabela's


## NewsBank Webscraper

In [3]:
# executable_path = "C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe"
# driver = webdriver.Chrome(executable_path=executable_path)

# base_url = "https://infoweb.newsbank.com/"
# search_url = base_url + "apps/news/advanced-search?p=AWNB&f=advanced"

# driver.get(search_url)

# driver.find_element_by_id("edit-val-base-0").send_keys("Apple")
# Select(driver.find_element_by_id("edit-fld-base-0")).select_by_visible_text("All Text")

# driver.find_element_by_id("edit-val-base-1").send_keys("2001-2008")
# Select(driver.find_element_by_id("edit-fld-base-1")).select_by_visible_text("Date(s)")

# driver.find_element_by_id("addTriplet").click()
# driver.find_element_by_id("edit-val-base-2").send_keys("SEC")
# Select(driver.find_element_by_id("edit-fld-base-2")).select_by_visible_text("All Text")

# driver.find_element_by_id("addTriplet").click()
# driver.find_element_by_id("edit-val-base-3").send_keys("audit")
# Select(driver.find_element_by_id("edit-fld-base-3")).select_by_visible_text("All Text")

# driver.find_element_by_id("addTriplet").click()
# driver.find_element_by_id("edit-val-base-4").send_keys("investigation")
# Select(driver.find_element_by_id("edit-fld-base-4")).select_by_visible_text("All Text")

# driver.find_element_by_id("addTriplet").click()
# driver.find_element_by_id("edit-val-base-5").send_keys("fraud")
# Select(driver.find_element_by_id("edit-fld-base-5")).select_by_visible_text("All Text")

# driver.find_element_by_id("edit-submit").click()

# # best match sorting
# driver.find_elements_by_xpath("//*[contains(text(), 'Best Match')]")[1].click()

In [4]:
executable_path = "C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe"
driver = webdriver.Chrome(executable_path=executable_path)

base_url   = "https://infoweb.newsbank.com"
news_url   = base_url + "/apps/news/document-view?p=AWNB"

data = {}
for i, row in comps.iterrows():
    
    comp = "\"" + row['search_term'].replace(" ", "%20") + "\""
    dates = str(row['fraud_date'].year) + '-' + str(row['date_release'].year)
    
    search_url = base_url + "/apps/news/results?p=AWNB&t=&sort=_rank_%3AD&maxresults=20&f=advanced"
    search_url += "&val-base-0=" + comp + "&fld-base-0=alltext"
    search_url += "&bln-base-1=and&val-base-1=" + dates + "&fld-base-1=YMD_date"
    search_url += "&bln-base-2=and&val-base-2=SEC&fld-base-2=alltext"
    search_url += "&bln-base-3=and&val-base-3=audit&fld-base-3=alltext"
    search_url += "&bln-base-4=and&val-base-4=investigation&fld-base-4=alltext"
    # search_url += "&bln-base-5=and&val-base-5=fraud&fld-base-5=alltext"
    
    driver.get(search_url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
     
    results = soup.find_all("article", {"id" : re.compile('search-hits__hit--*')})
    if not results:
        print('No results found for:', row['search_term'])
        data[(i,0)] = {
            'permno': row['permno'],
            'cusip': row['cusip'],
            'gvkey': row['gvkey'],
            'conm': row['conm'],
            'p_aaer': row['p_aaer'],
            'permno': row['permno'],
            'date_release': row['date_release'],
            'fraud_date': row['fraud_date'],
            'coname': row['coname'],
            'title' : None,
            'url' : None,
            'date' : None, 
            'source' : None
        }
    else:
        for j, result in enumerate(results):
            t = result.find("h3").text.replace('\nGo to the document viewer for ', '')
            u = base_url + result.find("h3").a['href']
            d = result.find("li", class_="search-hits__hit__meta__item search-hits__hit__meta__item--display-date").text.replace("\n", "").strip()
            s = result.find("li", class_="search-hits__hit__meta__item search-hits__hit__meta__item--source").text.replace("\n", "").strip()
            data[(i,j)] = {
                'permno': row['permno'],
                'cusip': row['cusip'],
                'gvkey': row['gvkey'],
                'conm': row['conm'],
                'p_aaer': row['p_aaer'],
                'permno': row['permno'],
                'date_release': row['date_release'],
                'fraud_date': row['fraud_date'],
                'coname': row['coname'],
                'title' : t,
                'url' : u,
                'date' : d, 
                'source' : s
            }
    
driver.close()
    
df = pd.DataFrame.from_dict(data).T
df['date'] = pd.to_datetime(df['date'])
df['url'] = df['url'].apply(lambda x: news_url + "&docref" + x.split('&docref')[1] if x else x)

mask = (df['date_release'] >= df['date']) & (df['fraud_date'] <= df['date']) | (df['date'].isnull())
df = df[mask]
df.sort_values(by=['coname', 'date'], inplace=True)

df.to_csv("data/fraud/fraud_news_search.csv", index=False)

No results found for: Geo Securities
No results found for: Interpublic Group of Companies
No results found for: Paccar
No results found for: Saba Software
No results found for: Stein Mart
No results found for: Volt Information Sciences
No results found for: WellCare Health Plans
No results found for: West Marine
